# 利用 Paddle Quantum 的 qchem 模块进行量子化学计算
_Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved._


qchem 是基于 Paddle Quantum 推出的用于量子化学研究的工具集。qchem 为量子化学领域的研究者提供了一系列工具，使他们可以利用量子计算方法完成量子化学任务。与此同时，qchem 也提供了方便开发者进行功能拓展的方式。目前，qchem 正处于开发之中，您可以将需求和建议通过 GitHub 的 issue 或 pull request 反馈给我们，我们会及时给出回复。

## 分子基态能量计算
qchem 为量子化学计算提供了很多便捷的工具。目前，qchem 模块支持下列分子波函数模版线路：
* Hardware Efficient ansatz[<sup>1</sup>](#refer-1),
* Hartree Fock ansatz[<sup>2</sup>](#refer-2),
* Unitary Coupled Cluster singles and doubles (UCCSD) ansatz[<sup>3</sup>](#refer-3).

让我们从具体的例子出发了解 qchem 的使用方法，下面我们演示了利用 qchem 求解氢分子基态的过程。

In [1]:
import paddle_quantum as pq
from paddle_quantum import qchem as pq_qchem
import warnings
warnings.filterwarnings("ignore")
import logging
logging.basicConfig(level=logging.INFO)

/home/zl/miniconda3/envs/pq/lib/python3.8/site-packages/openfermion/hamiltonians/hartree_fock.py:11: DeprecationWarning: Please use `OptimizeResult` from the `scipy.optimize` namespace, the `scipy.optimize.optimize` namespace is deprecated.
  from scipy.optimize.optimize import OptimizeResult
/home/zl/miniconda3/envs/pq/lib/python3.8/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:
/home/zl/miniconda3/envs/pq/lib/python3.8/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in 

接下来，我们会利用分子的一些主要性质，包括：分子的几何结构、分子的电荷、计算需要用到的量子化学基函数等，来构建一个 qchem 中的 `Molecule` 类。 

In [2]:
# `driver` 用来计算分子中的各种积分
driver = pq_qchem.PySCFDriver()

# 通过氢分子的性质构造一个 Molecule 类，注：长度单位为埃
mol = pq_qchem.Molecule(
    geometry=[("H", [0.0, 0.0, 0.0]), ("H", [0.0, 0.0, 0.74])],
    basis="sto-3g",
    multiplicity=1,   
    driver=driver
)

然后，我们需要为氢分子选择一种波函数模版线路。我们选择 `HardwareEfficient` 作为模版线路。

In [3]:
# 构建 HardwareEfficient 线路.
mol.build()
n_qubits = mol.num_qubits
depth = 2
cir = pq_qchem.HardwareEfficient(n_qubits, depth)

INFO:root:
#######################################
Molecule
#######################################
INFO:root:H2
INFO:root:Geometry:
INFO:root:H 0.00000, 0.00000, 0.00000
H 0.00000, 0.00000, 0.74000
INFO:root:Charge: 0
INFO:root:Multiplicity: 1
INFO:root:Unit: Angstrom
INFO:root:
#######################################
SCF Calculation (Classical)
#######################################
INFO:root:Basis: sto-3g


converged SCF energy = -1.11675930739643


INFO:root:SCF energy: -1.11676.


完成上面的步骤之后，我们可以调用 `GroundStateSolver` 求解器，并利用 PaddlePaddle 中的优化器来训练参数化量子线路。

In [4]:
print(type(mol))

<class 'paddle_quantum.qchem.molecule.Molecule'>


In [5]:
# 选择 paddlepaddle 中的 Adam 优化器
from paddle.optimizer import Adam

solver = pq_qchem.GroundStateSolver(Adam, num_iterations=100, tol=1e-5, save_every=10)
e, psi = solver.solve(cir, mol, learning_rate=0.5)

INFO:root:
#######################################
VQE (Ground State)
#######################################
INFO:root:Number of qubits: 4
INFO:root:Ansatz: HardwareEfficient
INFO:root:Optimizer: Adam
INFO:root:	learning_rate: 0.5
INFO:root:
Optimization:
INFO:root:	Itr 0, loss=-0.52172.
INFO:root:	Itr 10, loss=-1.05356.
INFO:root:	Itr 20, loss=-1.11600.
INFO:root:	Itr 30, loss=-1.12634.
INFO:root:	Itr 40, loss=-1.13320.
INFO:root:	Itr 50, loss=-1.13628.
INFO:root:Optimization converged after 58 iterations.
INFO:root:The final loss = -1.13636.


---
## 参考文献

[1] Kandala, Abhinav, et al. "Hardware-efficient variational quantum eigensolver for small molecules and quantum magnets." [Nature 549.7671 (2017): 242-246.](https://www.nature.com/articles/nature23879)

[2] Arute, Frank, et al. "Hartree-Fock on a superconducting qubit quantum computer." [Science 369.6507 (2020): 1084-1089.](https://www.science.org/doi/10.1126/science.abb9811)

[3] Abhinav, Aspuru-Guzik, et al. "A Quantum Computing View on Unitary Coupled Cluster Theory" (https://arxiv.org/abs/2109.15176)